<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

This notebook identifies the keywords to use for picking out crimes corresponding to a number of mandatory minimums in VA. It uses court data publicly available at https://virginiacourtdata.org/.

In [ ]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../../../..')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from recidiviz.calculator.modeling.population_projection.super_simulation.super_simulation_factory import SuperSimulationFactory
from recidiviz.calculator.modeling.population_projection.transition_table import TransitionTable
from recidiviz.calculator.modeling.population_projection.shell_compartment import ShellCompartment
from recidiviz.calculator.modeling.population_projection.spark_policy import SparkPolicy
from recidiviz.calculator.modeling.population_projection.utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)
pd.set_option('max_columns', 100)

In [ ]:
court_data = pd.concat([pd.read_csv(f'../state/VA/VA_data/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
                      
court_data = court_data.drop(
    ['HearingJury', 'HearingPlea', 'HearingRoom', 'fips', 'Locality', 'DrivingRestrictions', 
     'RestrictionEffectiveDate', 'RestrictionEndDate', 'RestitutionPaid', 
     'RestitutionAmount', 'Military', 'TrafficFatality', 'Sex', 'Race', 'Address', 'Costs', 
     'FinesCostPaid', 'ProgramType', 'CourtDMVSurrender', 'DriverImprovementClinic', 'VAAlcoholSafetyAction', 
     'OffenseDate', 'LifeDeath', 'OperatorLicenseSuspensionTime', 'FineAmount', 'ArrestDate',
     'AmendedCharge', 'AmendedCodeSection', 'AmendedChargeType', 'ConcludedBy', 'DispositionDate'], axis=1)
court_data = court_data[court_data.DispositionCode == 'Guilty'].drop('DispositionCode', axis=1)
print(f"number of records: {len(court_data)}")
court_data.head()

court_data = court_data[(court_data.HearingResult == 'Sent') | (court_data.HearingResult == 'Sentenced')]
court_data.SentenceTime = court_data.SentenceTime.fillna(0)
court_data.SentenceSuspended = court_data.SentenceSuspended.fillna(0)
court_data['effective_sentence_months'] = (court_data.SentenceTime - court_data.SentenceSuspended) / 365 * 12
court_data = court_data[court_data.Charge.notnull()]




In [ ]:
court_data['day'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[2]))
court_data['month'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[1]))
court_data['year'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[0]))

In [ ]:
def get_most_serious_case(df):
    most_serious_crime = df.sort_values('effective_sentence_months').iloc[-1]
    most_serious_crime['crimes'] = len(df)
#     most_serious_crime.DRUG_POSSESSION = df.DRUG_POSSESSION.any()
#     most_serious_crime.DRUG_SALE = df.DRUG_SALE.any()
#     most_serious_crime.DRUG_OTHER = df.DRUG_OTHER.any()
#     most_serious_crime.MARIJUANA = df.MARIJUANA.any()
    return most_serious_crime

court_data = court_data.groupby(['person_id', 'HearingDate'], as_index=False).apply(get_most_serious_case)

In [ ]:
mms_stats = pd.read_csv('../state/VA/2022/CurrentMMs.csv')
mms_stats = mms_stats[mms_stats.VCC.notnull()]
mms_stats['offense_code'] = mms_stats.VCC.apply(lambda x: x.split('-')[0] + x.split('-')[1])
mms_stats

In [ ]:
raw_va_sentence_df = pd.read_csv(
    '../state/VA/VA_data/unprocessed_va_historical_sentences_v2.csv',
    sep='\t'
)
raw_va_sentence_df['crime_type'] = raw_va_sentence_df['Offense Group'].ffill()
raw_va_sentence_df['offense_code'] = raw_va_sentence_df['VCC'].ffill()
raw_va_sentence_df['crime'] = raw_va_sentence_df['Off1VCC'].ffill()
raw_va_sentence_df['judge_id'] = raw_va_sentence_df['JudgeID'].ffill()
raw_va_sentence_df['sentence_type_code'] = raw_va_sentence_df['ActDisp'].ffill()
raw_va_sentence_df['effective_sentence_months'] = raw_va_sentence_df['effsent']
raw_va_sentence_df['fiscal_year'] = raw_va_sentence_df['FiscalYr'].ffill()
raw_va_sentence_df['life_sentence'] = raw_va_sentence_df['EffLif']
raw_va_sentence_df = raw_va_sentence_df.rename({'Off1Date':'offense_date'}, axis=1)

raw_va_sentence_df = raw_va_sentence_df[~raw_va_sentence_df['crime_type'].str.contains('Total')]


raw_va_sentence_df = raw_va_sentence_df.drop(
    ['Offense Group', 'VCC', 'Off1VCC', 'JudgeID', 'ActDisp', 'effsent', 'FiscalYr', 'EffLif'],
    axis=1
)
raw_va_sentence_df

In [ ]:
def count_instances(crime_code):
    return len(raw_va_sentence_df[raw_va_sentence_df.offense_code == crime_code])

mms_stats['instances'] = mms_stats.offense_code.apply(count_instances)
# plt.hist(mms_stats.instances / mms_stats.instances.sum(), bins=40)
# plt.yscale('log')
mms_stats.sort_values('instances', ascending=False)

In [ ]:
court_data['ASSAULT_ON_POLICE'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['LAW ENF', 'POLICE']]) and not any([i in x for i in [
        'ELUDE', 'FALSE', 'ELUDING'
    ]])
)

court_data['THIRD_DWI_10YR'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['3RD', '3+', 'THREE', 'THIRD']]) and not any([i in x for i in [
        'SHOPLIFT', 'A&B', 'LARC', 'ASSAULT', 'SHOPLFT', 'SCH I', 'SCH I', 'CONCEALMENT', 'PROTECTIVE', 
        'POSS', 'INDECENT EXPOSURE', 'DIST'
    ]])
)

court_data['DRUG_INTENT'] = court_data.Charge.apply(
    lambda x: any([i in x for i in [
        'INTENT',
        'W/INT',
        'PWI']]) 
    and not any([i in x for i in [
        'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL', 'CIG']])
)
court_data['DRUG_SALE'] = court_data.Charge.apply(
    lambda x: any([i in x for i in [
        'DISTRB',
        'DISTRIBUTION',
        'DISTRIB',
        'DIST',
        'PWID', 
        'SALE',
        'SELL']])
        and not any([i in x for i in [
        'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL', 'CIG']])
)
court_data['2ND'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['2ND', 'SECOND']])
)
court_data['SECOND_PWID'] = (court_data['DRUG_INTENT'] | court_data['DRUG_SALE']) & court_data['2ND']

court_data = court_data.drop(
    ['DRUG_INTENT', 'DRUG_SALE', '2ND'], axis=1)
# court_data['DRUG'] = court_data.Charge.apply(
#     lambda x: any([i in x for i in [
#         'POSS',
#         'CONTROLLED SUBSTANCE',
#         'SCH 1',
#         'DISTRB',
#         'DISTRIB',
#         'DISTRIBUTION',
#         'SELL'
#         'PWID', 
#         'MARIJUANA',
#         'MARIJ',
#         'COCAINE',
#         'HEROIN',
#         'DRUG'
#     ]]) and not any([i in x for i in [
#         'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL'
#     ]])
# )

# court_data.DRUG_POSSESSION = court_data.DRUG_POSSESSION & court_data.DRUG


# court_data.DRUG_SALE = court_data.DRUG_SALE & court_data.DRUG

# court_data['DRUG_OTHER'] = court_data.DRUG & ~(court_data.DRUG_SALE | court_data.DRUG_POSSESSION)



court_data[court_data.SECOND_PWID].groupby('Charge').count().sort_values('HearingType', ascending=False).iloc[:200]

In [ ]:
nar_data = court_data[court_data.SECOND_PWID]
nar_data[(nar_data.year == 2017) | (nar_data.year == 2018) | (nar_data.year == 2019)]
nar_data